## Imports

In [1]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import regex as re
import pandas as pd
import numpy as np
import random 
import pycountry
np.random.seed = 42
random.seed(42)

pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 10)

## Data Format

In [2]:
mailchimp_data = pd.read_csv('./data/MailChimp cleaned records headers.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 10)
mailchimp_data

,Email Address,First Name,Last Name,Board Member,Gender,Chapter,Reunion Year,Country,Degree,MEMBER_RATING,OPTIN_TIME,OPTIN_IP,CONFIRM_TIME,CONFIRM_IP,LATITUDE,LONGITUDE,GMTOFF,DSTOFF,TIMEZONE,CC,REGION,CLEAN_TIME,CLEAN_CAMPAIGN_TITLE,CLEAN_CAMPAIGN_ID,LEID,EUID,NOTES,TAGS


The mailchimp dataset comes from the salesforce dataset. This Cleaned dataset would only include emails that have bounced for one reason or another. It will not include any members who have opted out of the email service or those that are receiving emails without any complications. It is important to note that updating information within salesforce for those who have opted out may re-instantiate the email services so we are specifically working with cleaned/bounced records. Additionally, the datasets created for those who are Subscribed and Unsubscribed will have different column names than the Cleaned dataset here (ie Clean_Time, Clean_campaign_title, ect). Therefore any function created here may not work for those datasets. 

In [3]:
saa_pride_data = pd.read_excel('./data/SAA Pride member reports headings.xlsx')
saa_pride_data.reset_index(inplace = True)
saa_pride_data

,index,pref_mail_name,pref_class_year,home_city,home_state_code,home_country,home_phone_area_code,home_phone_number,home_email_address,bus_city,bus_state_code,bus_country,bus_phone_area_code,bus_phone_number,bus_email_address,first_name,last_name,pref_name_sort,email_switch,saa_email_address,gsb_email_address,other_email_address,pref_phone_area_code,pref_phone_number,pref_phone_addr_type,memb_status_desc,short_degree_string,parent_degree_string,short_degree_string_spouse,parent_degree_string_spouse,primary_sort_name,plan_name,primary_ind


The Stanford Alumni Association has it's own dataset that may or may not have additional or more recent data on some of the members. It may also have outdated data. Students are given an email but when they become alumni the email needs to be updated. Whether it is updated to specifically an 'alumni.stanford.edu' address or to another would be at the students discrepancy and isn't always updated.

## Helper Functions

In [4]:
def removeEmailDomain(email):
    email = str(email)
    #if email string contains @ symbol return the handle
    if email.find('@')>-1:
        return email.split('@')[0]
    else:
        return 'n/a'

In [5]:
def mapCountry(countryStr):
    if countryStr not in [np.nan, None, '*', '','N/A']:
        return pycountry.countries.search_fuzzy(countryStr)[0].alpha_3
    return 'n/a'

In [6]:
def ohe(df, column):    
    for col in column:
        train = df[[col]]
        ohe = OneHotEncoder(sparse=False, handle_unknown="error")
        ohe.fit(train)
        encoded_train = ohe.transform(train)
        col_names = [f"{col}_{f}" for f in ohe.get_feature_names()]
        encoded_train = pd.DataFrame(encoded_train,
                                     columns=col_names, index=df.index)
        df = pd.concat([df, encoded_train], axis=1)
        
    return df

In [7]:
def parseDegreeColumns(df,deg_colName):
    vectorizer = CountVectorizer()
    # Replace numbers and special characters with ' '
    t = vectorizer.fit_transform(df[deg_colName].apply(lambda x: re.sub('[;,\'*0-9]', ' ', x)))
    # Remove original degree column from original dataframe
    df = df.drop(columns=[deg_colName])
    # Create new vectorized degree columns and concatenate
    deg_df =  pd.DataFrame(t.toarray(),columns=vectorizer.get_feature_names_out())
    if 'nan' in deg_df.columns:
        deg_df.drop(columns = 'nan')
    return pd.concat([df,deg_df],axis = 1)

### Pokemon Data

In [8]:
df_saa = pd.read_excel('./data/SAA_Pokemon_FakeDB.xlsx')
df_saa


,pref_mail_name,pref_class_year,home_city,home_state_code,home_country,home_phone_area_code,home_phone_number,home_email_address,bus_city,bus_state_code,bus_country,bus_phone_area_code,bus_phone_number,bus_email_address,first_name,last_name,pref_name_sort,email_switch,saa_email_address,gsb_email_address,other_email_address,pref_phone_area_code,pref_phone_number,pref_phone_addr_type,memb_status_desc,short_degree_string,parent_degree_string,short_degree_string_spouse,parent_degree_string_spouse,primary_sort_name,plan_name,primary_ind
0,NaN,2004.0,Shanghai,NaN,*,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Growlithe,Ice,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Madrid,NaN,China,NaN,NaN,weedleg4046@stanfordalumni.org,NaN,NaN,China,NaN,NaN,NaN,Weedle,Grass,NaN,NaN,w.grass5053@alumni.stanford.edu,NaN,NaN,NaN,NaN,NaN,NaN,'82,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Seoul,NaN,Kuwait,NaN,NaN,aerodactyl.electric2974@alumni.stanford.edu,NaN,NaN,Kuwait,NaN,NaN,NaN,Aerodactyl,Electric,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,London,NaN,NaN,NaN,*,*,NaN,NaN,Japan,NaN,NaN,NaN,Pinsir,Fire,NaN,pinsirfire4582@gmail.com,*,NaN,NaN,NaN,NaN,NaN,NaN,"JD '94, PhD '97",NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,London,NaN,USA,NaN,775 0678-214,*,NaN,NaN,NaN,NaN,NaN,NaN,Horsea,Ice,NaN,hice7313@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,NaN,NaN,Boston,MA,Japan,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,Smeargle,Electric,NaN,smeargleelectric9444@gmail.com,*,NaN,NaN,NaN,NaN,NaN,NaN,"PhD '96, MBA '87",NaN,NaN,NaN,NaN,NaN,NaN
3996,NaN,2000.0,Shanghai,NaN,NaN,NaN,*,kabutops.steel1285@stanfordalumni.org,NaN,NaN,China,NaN,NaN,NaN,Kabutops,Steel,NaN,k.steel5317@alumni.stanford.edu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BS '93,NaN,NaN,NaN,NaN,NaN,NaN
3997,NaN,NaN,NaN,NaN,Kuwait,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,Slowking,Dragon,NaN,NaN,slowkingd5563@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,"PhD '84, JD '93",NaN,NaN,NaN,NaN,NaN,NaN
3998,NaN,NaN,NaN,NaN,Japan,NaN,NaN,larvitar.electric9778@stanfordalumni.org,NaN,NaN,United States,NaN,NaN,NaN,Larvitar,Electric,NaN,l.electric7920@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MBA '92, '98, JD '86",NaN,NaN,NaN,NaN,NaN,NaN


# Preprocess SAA dataframe

## Filter Columns

In [9]:
# Filter necessary columns
df_saa = df_saa.filter(['first_name', 'last_name',
                        'home_country', 'home_email_address', 'short_degree_string',
                        'bus_email_address', 'bus_country', 'email_switch',
                        'saa_email_address', 'gsb_email_address', 'other_email_address'])
df_saa

,first_name,last_name,home_country,home_email_address,short_degree_string,bus_email_address,bus_country,email_switch,saa_email_address,gsb_email_address,other_email_address
0,Growlithe,Ice,*,NaN,NaN,NaN,NaN,NaN,*,NaN,NaN
1,Weedle,Grass,China,weedleg4046@stanfordalumni.org,'82,NaN,China,NaN,w.grass5053@alumni.stanford.edu,NaN,NaN
2,Aerodactyl,Electric,Kuwait,aerodactyl.electric2974@alumni.stanford.edu,NaN,NaN,Kuwait,NaN,*,NaN,NaN
3,Pinsir,Fire,NaN,*,"JD '94, PhD '97",NaN,Japan,pinsirfire4582@gmail.com,*,NaN,NaN
4,Horsea,Ice,USA,*,NaN,NaN,NaN,hice7313@stanfordalumni.org,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3995,Smeargle,Electric,Japan,*,"PhD '96, MBA '87",NaN,NaN,smeargleelectric9444@gmail.com,*,NaN,NaN
3996,Kabutops,Steel,NaN,kabutops.steel1285@stanfordalumni.org,BS '93,NaN,China,k.steel5317@alumni.stanford.edu,NaN,NaN,NaN
3997,Slowking,Dragon,Kuwait,*,"PhD '84, JD '93",NaN,NaN,NaN,slowkingd5563@stanfordalumni.org,NaN,NaN
3998,Larvitar,Electric,Japan,larvitar.electric9778@stanfordalumni.org,"MBA '92, '98, JD '86",NaN,United States,l.electric7920@stanfordalumni.org,NaN,NaN,NaN


## Strip and lower case

In [10]:
for col in df_saa.columns:    
    df_saa[col] = df_saa[col].astype(str).str.lower().str.strip()
df_saa

,first_name,last_name,home_country,home_email_address,short_degree_string,bus_email_address,bus_country,email_switch,saa_email_address,gsb_email_address,other_email_address
0,growlithe,ice,*,nan,nan,nan,nan,nan,*,nan,nan
1,weedle,grass,china,weedleg4046@stanfordalumni.org,'82,nan,china,nan,w.grass5053@alumni.stanford.edu,nan,nan
2,aerodactyl,electric,kuwait,aerodactyl.electric2974@alumni.stanford.edu,nan,nan,kuwait,nan,*,nan,nan
3,pinsir,fire,nan,*,"jd '94, phd '97",nan,japan,pinsirfire4582@gmail.com,*,nan,nan
4,horsea,ice,usa,*,nan,nan,nan,hice7313@stanfordalumni.org,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...
3995,smeargle,electric,japan,*,"phd '96, mba '87",nan,nan,smeargleelectric9444@gmail.com,*,nan,nan
3996,kabutops,steel,nan,kabutops.steel1285@stanfordalumni.org,bs '93,nan,china,k.steel5317@alumni.stanford.edu,nan,nan,nan
3997,slowking,dragon,kuwait,*,"phd '84, jd '93",nan,nan,nan,slowkingd5563@stanfordalumni.org,nan,nan
3998,larvitar,electric,japan,larvitar.electric9778@stanfordalumni.org,"mba '92, '98, jd '86",nan,united states,l.electric7920@stanfordalumni.org,nan,nan,nan


## Remove domains from each email. Retain handles

In [11]:
email_cols = ['home_email_address', 'bus_email_address', 'email_switch',
              'saa_email_address', 'gsb_email_address', 'other_email_address']
for email in email_cols:
    df_saa[email] = df_saa[email].apply(removeEmailDomain)
df_saa

,first_name,last_name,home_country,home_email_address,short_degree_string,bus_email_address,bus_country,email_switch,saa_email_address,gsb_email_address,other_email_address
0,growlithe,ice,*,n/a,nan,n/a,nan,n/a,n/a,n/a,n/a
1,weedle,grass,china,weedleg4046,'82,n/a,china,n/a,w.grass5053,n/a,n/a
2,aerodactyl,electric,kuwait,aerodactyl.electric2974,nan,n/a,kuwait,n/a,n/a,n/a,n/a
3,pinsir,fire,nan,n/a,"jd '94, phd '97",n/a,japan,pinsirfire4582,n/a,n/a,n/a
4,horsea,ice,usa,n/a,nan,n/a,nan,hice7313,n/a,n/a,n/a
...,...,...,...,...,...,...,...,...,...,...,...
3995,smeargle,electric,japan,n/a,"phd '96, mba '87",n/a,nan,smeargleelectric9444,n/a,n/a,n/a
3996,kabutops,steel,nan,kabutops.steel1285,bs '93,n/a,china,k.steel5317,n/a,n/a,n/a
3997,slowking,dragon,kuwait,n/a,"phd '84, jd '93",n/a,nan,n/a,slowkingd5563,n/a,n/a
3998,larvitar,electric,japan,larvitar.electric9778,"mba '92, '98, jd '86",n/a,united states,l.electric7920,n/a,n/a,n/a


## Replace nulls

In [12]:
df_saa.fillna('n/a')

,first_name,last_name,home_country,home_email_address,short_degree_string,bus_email_address,bus_country,email_switch,saa_email_address,gsb_email_address,other_email_address
0,growlithe,ice,*,n/a,nan,n/a,nan,n/a,n/a,n/a,n/a
1,weedle,grass,china,weedleg4046,'82,n/a,china,n/a,w.grass5053,n/a,n/a
2,aerodactyl,electric,kuwait,aerodactyl.electric2974,nan,n/a,kuwait,n/a,n/a,n/a,n/a
3,pinsir,fire,nan,n/a,"jd '94, phd '97",n/a,japan,pinsirfire4582,n/a,n/a,n/a
4,horsea,ice,usa,n/a,nan,n/a,nan,hice7313,n/a,n/a,n/a
...,...,...,...,...,...,...,...,...,...,...,...
3995,smeargle,electric,japan,n/a,"phd '96, mba '87",n/a,nan,smeargleelectric9444,n/a,n/a,n/a
3996,kabutops,steel,nan,kabutops.steel1285,bs '93,n/a,china,k.steel5317,n/a,n/a,n/a
3997,slowking,dragon,kuwait,n/a,"phd '84, jd '93",n/a,nan,n/a,slowkingd5563,n/a,n/a
3998,larvitar,electric,japan,larvitar.electric9778,"mba '92, '98, jd '86",n/a,united states,l.electric7920,n/a,n/a,n/a


## Parse Degree string

In [13]:
parseDegreeColumns(df_saa,deg_colName='short_degree_string')

,first_name,last_name,home_country,home_email_address,bus_email_address,bus_country,email_switch,saa_email_address,gsb_email_address,other_email_address,ba,bs,jd,ma,mba,md,ms,nan,phd
0,growlithe,ice,*,n/a,n/a,nan,n/a,n/a,n/a,n/a,0,0,0,0,0,0,0,1,0
1,weedle,grass,china,weedleg4046,n/a,china,n/a,w.grass5053,n/a,n/a,0,0,0,0,0,0,0,0,0
2,aerodactyl,electric,kuwait,aerodactyl.electric2974,n/a,kuwait,n/a,n/a,n/a,n/a,0,0,0,0,0,0,0,1,0
3,pinsir,fire,nan,n/a,n/a,japan,pinsirfire4582,n/a,n/a,n/a,0,0,1,0,0,0,0,0,1
4,horsea,ice,usa,n/a,n/a,nan,hice7313,n/a,n/a,n/a,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,smeargle,electric,japan,n/a,n/a,nan,smeargleelectric9444,n/a,n/a,n/a,0,0,0,0,1,0,0,0,1
3996,kabutops,steel,nan,kabutops.steel1285,n/a,china,k.steel5317,n/a,n/a,n/a,0,1,0,0,0,0,0,0,0
3997,slowking,dragon,kuwait,n/a,n/a,nan,n/a,slowkingd5563,n/a,n/a,0,0,1,0,0,0,0,0,1
3998,larvitar,electric,japan,larvitar.electric9778,n/a,united states,l.electric7920,n/a,n/a,n/a,0,0,1,0,1,0,0,0,0


In [25]:
df_saa.home_country.apply(mapCountry)

KeyboardInterrupt: 

# Mail Chimp Dataframe

In [14]:
df_mailchimp = pd.read_csv('./data/Fake_MailChimp_cleaned_Pokemon.csv')
df_mailchimp

,Email Address,First Name,Last Name,Board Member,Gender,Chapter,Reunion Year,Country,Degree,MEMBER_RATING,OPTIN_TIME,OPTIN_IP,CONFIRM_TIME,CONFIRM_IP,LATITUDE,LONGITUDE,GMTOFF,DSTOFF,TIMEZONE,CC,REGION,CLEAN_TIME,CLEAN_CAMPAIGN_TITLE,CLEAN_CAMPAIGN_ID,LEID,EUID,NOTES,TAGS
0,slakoth.normal3945@gmail.com,Slakoth,Normal,False,NaN,Texas,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e.rock7454@gmail.com,Espeon,Rock,True,F,DC Area,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rhydonghost7966@alumni.stanford.edu,Rhydon,Ghost,False,M,Bay Area,NaN,USA,MBA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,porygong9247@stanfordalumni.org,Porygon,Grass,False,M,Bay Area,NaN,Japan,MS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tangelagrass1376@gmail.com,Tangela,Grass,False,NaN,New England,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,c.electric7518@gmail.com,Chansey,Steel,True,F,Other US,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,blissey.ghost4154@gmail.com,Blissey,Ghost,False,M,New England,NaN,Macao Special Administrative Region of China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Preprocess mailchimp dataframe

## Filter necessary columns

In [15]:
# Filter necessary columns
df_mailchimp = df_mailchimp.filter(['First Name', 'Last Name', 'Email Address',
                                         'Degree', 'Country'])
df_mailchimp

,First Name,Last Name,Email Address,Degree,Country
0,Slakoth,Normal,slakoth.normal3945@gmail.com,NaN,USA
1,Espeon,Rock,e.rock7454@gmail.com,NaN,United States
2,Rhydon,Ghost,rhydonghost7966@alumni.stanford.edu,MBA,USA
3,Porygon,Grass,porygong9247@stanfordalumni.org,MS,Japan
4,Tangela,Grass,tangelagrass1376@gmail.com,NaN,United States
5,Chansey,Steel,c.electric7518@gmail.com,NaN,USA
6,Blissey,Ghost,blissey.ghost4154@gmail.com,NaN,Macao Special Administrative Region of China


## Strip and lowercase all names and emails

In [16]:
for col in ['First Name','Last Name','Email Address','Degree']:
    df_mailchimp[col] = df_mailchimp[col].str.lower().str.strip()

In [17]:
df_mailchimp

,First Name,Last Name,Email Address,Degree,Country
0,slakoth,normal,slakoth.normal3945@gmail.com,NaN,USA
1,espeon,rock,e.rock7454@gmail.com,NaN,United States
2,rhydon,ghost,rhydonghost7966@alumni.stanford.edu,mba,USA
3,porygon,grass,porygong9247@stanfordalumni.org,ms,Japan
4,tangela,grass,tangelagrass1376@gmail.com,NaN,United States
5,chansey,steel,c.electric7518@gmail.com,NaN,USA
6,blissey,ghost,blissey.ghost4154@gmail.com,NaN,Macao Special Administrative Region of China


## Remove email domain

In [18]:
df_mailchimp['Email Address'] = df_mailchimp['Email Address'].apply(removeEmailDomain)
df_mailchimp

,First Name,Last Name,Email Address,Degree,Country
0,slakoth,normal,slakoth.normal3945,NaN,USA
1,espeon,rock,e.rock7454,NaN,United States
2,rhydon,ghost,rhydonghost7966,mba,USA
3,porygon,grass,porygong9247,ms,Japan
4,tangela,grass,tangelagrass1376,NaN,United States
5,chansey,steel,c.electric7518,NaN,USA
6,blissey,ghost,blissey.ghost4154,NaN,Macao Special Administrative Region of China


## Convert Country to 3 digit code

In [19]:
df_mailchimp.Country = df_mailchimp.Country.apply(mapCountry)
df_mailchimp

,First Name,Last Name,Email Address,Degree,Country
0,slakoth,normal,slakoth.normal3945,NaN,USA
1,espeon,rock,e.rock7454,NaN,USA
2,rhydon,ghost,rhydonghost7966,mba,USA
3,porygon,grass,porygong9247,ms,JPN
4,tangela,grass,tangelagrass1376,NaN,USA
5,chansey,steel,c.electric7518,NaN,USA
6,blissey,ghost,blissey.ghost4154,NaN,MAC


## Replace nulls with 'n/a'

In [20]:
df_mailchimp.fillna('n/a',inplace=True)
df_mailchimp

,First Name,Last Name,Email Address,Degree,Country
0,slakoth,normal,slakoth.normal3945,n/a,USA
1,espeon,rock,e.rock7454,n/a,USA
2,rhydon,ghost,rhydonghost7966,mba,USA
3,porygon,grass,porygong9247,ms,JPN
4,tangela,grass,tangelagrass1376,n/a,USA
5,chansey,steel,c.electric7518,n/a,USA
6,blissey,ghost,blissey.ghost4154,n/a,MAC


## Parse Degree String

In [21]:
parseDegreeColumns(df_mailchimp,deg_colName='Degree')

,First Name,Last Name,Email Address,Country,mba,ms
0,slakoth,normal,slakoth.normal3945,USA,0,0
1,espeon,rock,e.rock7454,USA,0,0
2,rhydon,ghost,rhydonghost7966,USA,1,0
3,porygon,grass,porygong9247,JPN,0,1
4,tangela,grass,tangelagrass1376,USA,0,0
5,chansey,steel,c.electric7518,USA,0,0
6,blissey,ghost,blissey.ghost4154,MAC,0,0


In [22]:
test = df_mailchimp

In [23]:
ohe(test,['First Name'])

C:\Users\saads\anaconda3\envs\tfGPU\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,First Name,Last Name,Email Address,Degree,Country,First Name_x0_blissey,First Name_x0_chansey,First Name_x0_espeon,First Name_x0_porygon,First Name_x0_rhydon,First Name_x0_slakoth,First Name_x0_tangela
0,slakoth,normal,slakoth.normal3945,n/a,USA,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,espeon,rock,e.rock7454,n/a,USA,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,rhydon,ghost,rhydonghost7966,mba,USA,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,porygon,grass,porygong9247,ms,JPN,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,tangela,grass,tangelagrass1376,n/a,USA,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,chansey,steel,c.electric7518,n/a,USA,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,blissey,ghost,blissey.ghost4154,n/a,MAC,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
saa_poke2.fillna(value='Not Available', inplace=True)
emails = ['home_email_address', 'bus_email_address', 'email_switch', 'saa_email_address',\
         'gsb_email_address', 'other_email_address']
handles = []
for x in emails:
    for i in range(0,saa_poke2.shape[0]):
        if '@' in saa_poke2[x][i]:
            saa_poke2[x+'_handle'] = saa_poke2[x].str.split('@').str[0]
        else:
            saa_poke2[x+'_handle'] = saa_poke2[x]


NameError: name 'saa_poke2' is not defined

In [ ]:
emails

In [ ]:
handles = []
for i in range(0,len(emails)):
    handles.append(emails[i]+'_handle')
handles

## Function

In [ ]:
results_dict = {}
for i in range(0,mailchimp_poke2.shape[1]+1):
    target = mailchimp_poke2.iloc[i]
    target_dict = {'first_name': [target[1]], 'last_name': [target[2]],\
               emails[0]: [target[0]],\
               emails[1]: [target[0]],\
               emails[2]: [target[0]],\
               emails[3]: [target[0]],\
               emails[4]: [target[0]],\
               emails[5]: [target[0]],\
                   handles[0]: [target[4]],\
                   handles[1]: [target[4]],\
                   handles[2]: [target[4]],\
                   handles[3]: [target[4]],\
                   handles[4]: [target[4]],\
                   handles[5]: [target[4]],\
               'home_country': [target[3]]}
    df = pd.DataFrame.from_dict(target_dict)
    subset_saa = saa_poke2[saa_poke2['first_name'] == df.loc[0,'first_name']] 
    #the 0 is calling for row, so it does not return a series
    subset_saa_new = pd.concat([df,subset_saa], axis = 0)
    ohe_df = ohe(subset_saa_new, subset_saa_new.columns)
    ohe_df.drop(columns = subset_saa_new.columns, inplace = True)
    y = np.array(ohe_df.iloc[0])
    y = y.reshape(1,-1)
    cos_sim = cosine_similarity(ohe_df, y)
    cos_sim = pd.DataFrame(data=cos_sim, index=ohe_df.index).sort_values(by=0, ascending=False) #[1:]
    results = list(cos_sim.index)
    results_df = subset_saa_new.loc[results]
    results_dict[i] = results_df

In [ ]:
target[4]

In [ ]:
subset_saa_new

In [ ]:
mailchimp_poke2

In [ ]:
name_first = input('First Name: ')

results_dict[mailchimp_poke2[mailchimp_poke2['First Name']
                             == name_first].index[0]].head(5)

These are the 5 most likely matches.

## Next Steps

Next step would be to take a proactive approach to mitigate the number of emails that bounce in the future. We would suggest using the salesforce dataset to check for recent grads to reach out before they lose their student emails and ask for updated contact information and their subsequent plans after graduation. It would be easier to update records proactively when we still have accurate contact information. For subsequent plans after graduation, this would be to keeping location information of members to more accurately send regional events and functions. 